In [6]:
import pandas as pd 
# df =pd.read_csv("ratings.csv")

df = pd.io.parsers.read_csv('dataset/ratings.dat',
    names=['userid', 'movieid', 'ratings'],encoding='latin-1',
    engine='python', delimiter='::')

In [7]:
df

,userid,movieid,ratings
1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
...,...,...,...
6040,1091,1,956716541
6040,1094,5,956704887
6040,562,5,956704746
6040,1096,4,956715648


In [10]:
movie_data = pd.io.parsers.read_csv('movies.dat',
    names=['movie_id', 'title', 'genre'],encoding='latin-1',
    engine='python', delimiter='::')

In [11]:
movie_data

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [21]:
import numpy as np
ratings_mat = np.ndarray(
    shape=(np.max(data.movie_id.values), np.max(data.user_id.values)),
    dtype=np.uint8)
ratings_mat[data.movie_id.values-1, data.user_id.values-1] = data.rating.values

In [22]:
ratings_mat

array([[5, 0, 0, ..., 0, 0, 3],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [24]:
ratings_mat.shape

(3952, 6040)

In [25]:
normalised_mat = ratings_mat - np.asarray([(np.mean(ratings_mat, 1))]).T

In [26]:
normalised_mat

array([[ 3.57400662, -1.42599338, -1.42599338, ..., -1.42599338,
        -1.42599338,  1.57400662],
       [-0.37152318, -0.37152318, -0.37152318, ..., -0.37152318,
        -0.37152318, -0.37152318],
       [-0.23874172, -0.23874172, -0.23874172, ..., -0.23874172,
        -0.23874172, -0.23874172],
       ...,
       [-0.03278146, -0.03278146, -0.03278146, ..., -0.03278146,
        -0.03278146, -0.03278146],
       [-0.02582781, -0.02582781, -0.02582781, ..., -0.02582781,
        -0.02582781, -0.02582781],
       [-0.24288079, -0.24288079, -0.24288079, ..., -0.24288079,
        -0.24288079, -0.24288079]])

In [27]:
A = normalised_mat.T / np.sqrt(ratings_mat.shape[0] - 1)
U, S, V = np.linalg.svd(A)

In [28]:
def top_cosine_similarity(data, movie_id, top_n=10):
    index = movie_id - 1 # Movie id starts from 1 in the dataset
    movie_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

In [29]:
def print_similar_movies(movie_data, movie_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    movie_data[movie_data.movie_id == movie_id].title.values[0]))
    for id in top_indexes + 1:
        print(movie_data[movie_data.movie_id == id].title.values[0])

In [32]:
k = 50
movie_id = 3793 # (getting an id from movies.dat)
top_n = 10
sliced = V.T[:, :k] # representative data
indexes = top_cosine_similarity(sliced, movie_id, top_n)
# Print the top N similar movies.

print_similar_movies(movie_data, movie_id, indexes)

Recommendations for X-Men (2000): 

X-Men (2000)
Shaft (2000)
Romeo Must Die (2000)
Titan A.E. (2000)
Shanghai Noon (2000)
Gone in 60 Seconds (2000)
Gladiator (2000)
Mission: Impossible 2 (2000)
Battlefield Earth (2000)
Highlander: Endgame (2000)


C:\Users\Aaditya\AppData\Local\Temp\ipykernel_26820\3890564185.py:5: RuntimeWarning: invalid value encountered in divide
  similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
